#install dependencies

In [1]:
#install all packages needed
!pip install opencv-python numpy scikit-image scikit-learn tensorflow transformers

import os
import random
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns

import tensorflow as tf
import torch
import torchvision.transforms as transforms
from skimage.feature import graycomatrix, graycoprops
from transformers import ViTFeatureExtractor, ViTModel
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif


#declare function

In [2]:
#function untuk augmentation (supaya dataset gambar lebih variatif, dia di flip vertical, horizontal sama brightness nya dirandom)

def load_images(directory, label, augment_count=3):
    images = []
    labels = []

    transform_aug = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.3, contrast=0.3),
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  #supaya kompatible semua jenis file gambar
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path).convert("RGB")

            #resize gambar supaya semua gambar sama ukurannya
            img_resized = transforms.Resize((224, 224))(img)
            images.append(np.array(img_resized))
            labels.append(label)

            #looping untuk pengaplikasian proses augmentation
            for _ in range(augment_count):
                img_aug = transform_aug(img)
                images.append(np.transpose(img_aug.numpy(), (1, 2, 0)))
                labels.append(label)

    return images, labels


In [3]:
#load dataset
classes = ['Healthy', 'Mosaic', 'RedRot', 'Rust', 'Yellow'] #define class dari imagenya
images, labels = [], [] #buat list buat store semua images sama label nya

for idx, cls in enumerate(classes): #iterasi masukkin semua gambar sama class ke list
    imgs, lbls = load_images(f'/content/drive/MyDrive/dataset/{cls}', idx)
    images.extend(imgs)
    labels.extend(lbls)

#shuffle dataset
combined = list(zip(images, labels)) #gabungin dulu gambar sama labelnya
random.shuffle(combined)
images[:], labels[:] = zip(*combined) #setelah shuffle kita pisahin lagi gambar sama labelnya

#Feature extraction

In [4]:
#glcm feature extraction ini untuk nanti ektrak fitur tekstur dari objek dengan menghitung kedekatan antar piksel

def extract_glcm_features(image):
    img_uint8 = (image * 255).astype(np.uint8)  #konversi dari float64 ke uint8 (supaya kompatibel)
    gray = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2GRAY) #konversi ke grayscale
    glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    return [ #buat return semua value -> contrast, energy, correlation dan homogeneity
        graycoprops(glcm, 'contrast')[0, 0],
        graycoprops(glcm, 'energy')[0, 0],
        graycoprops(glcm, 'correlation')[0, 0],
        graycoprops(glcm, 'homogeneity')[0, 0]
    ]



In [5]:
#ekstrak statistik warna
def extract_color_stats(image):
    image = image / 255.0
    r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]  #gambar dipisah jadi data rgb (red, green, blue)
    return [r.mean(), r.std(), g.mean(), g.std(), b.mean(), b.std()] #return statistik warna

In [6]:
#ekstraksi fitur vit
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #panggil model vit yang mau dipakai
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)

def extract_vit_features(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)), #resize ke input vit
        transforms.ToTensor(),
        transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std) #normalisasi gambar
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = vit_model(image_tensor).last_hidden_state[:, 0, :].cpu().numpy()  # ambil CLS token
    return features.flatten()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [7]:
#ekstraksi semua fitur (glcm, color dan vit)
def extract_features(image):
    return np.concatenate([
        extract_glcm_features(image),
        extract_color_stats(image),
        extract_vit_features(image)
    ])


In [10]:
#store semua features ke list
features = np.array([extract_features(img) for img in tqdm(images)], dtype=np.float32)
y = np.array(labels)


100%|██████████| 10084/10084 [03:22<00:00, 49.90it/s]


In [11]:
# Save and reuse
np.save("features.npy", features)
np.save("labels.npy", y)

# Load features
features = np.load("features.npy")
y = np.load("labels.npy")

In [12]:
# dari semua fitur, kita hanya gunakan 300 fitur yang terbaik
selector = SelectKBest(score_func=f_classif, k=400)
X_selected = selector.fit_transform(features, y)

#  split dataset
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, stratify=y, random_state=42)

# normlisasi
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#Training model

In [13]:
#training svm (svc linear kernel)
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

#evaluasi model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9410
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       418
           1       0.94      0.93      0.94       370
           2       0.93      0.93      0.93       414
           3       0.98      0.95      0.97       411
           4       0.92      0.92      0.92       404

    accuracy                           0.94      2017
   macro avg       0.94      0.94      0.94      2017
weighted avg       0.94      0.94      0.94      2017



In [14]:
#training dengan gridseachcv
param_grid = { #declare parameter yang akan dipakai
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.001, 0.01, 0.1, 'scale', 'auto']
}
#training svc kernel linear tambah gridsearchcv supaya yang digunakan adalah parameter terbaik dnegan custom parameter, tidak pakai parameter svc default
grid = GridSearchCV(SVC(), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_scaled, y_train)

clf = grid.best_estimator_
y_pred = clf.predict(X_test_scaled)

#evaluasi model
print("Best Parameters:", grid.best_params_)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred, target_names=classes))

Best Parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Accuracy: 0.9717
              precision    recall  f1-score   support

     Healthy       0.96      0.99      0.98       418
      Mosaic       0.98      0.97      0.97       370
      RedRot       0.96      0.97      0.97       414
        Rust       1.00      0.97      0.99       411
      Yellow       0.96      0.96      0.96       404

    accuracy                           0.97      2017
   macro avg       0.97      0.97      0.97      2017
weighted avg       0.97      0.97      0.97      2017

